In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [4]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# Decompose using statsmodels (as before)
from statsmodels.tsa.seasonal import seasonal_decompose

# Make sure to replace 'path_to_your_data.csv' with the actual path to your dataset.
df = pd.read_csv('/content/avocado_processed (1).csv')

# Data preprocessing
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

df = df.set_index('date')

# Resample the data to monthly frequency and sum the 'total_volume'
monthly_sales = df['total_volume'].resample('M').sum()
decomposition = seasonal_decompose(monthly_sales, model='additive', period=12)
# Create a Dash application instance
app = dash.Dash(__name__)

# Layout of the Dash application
app.layout = html.Div(style={'backgroundColor': '#1E1E1E', 'color': 'white', 'padding': '50px'}, children=[
    html.H1(children='Sales Forecasting Dashboard'),
    html.Hr(style={'border': '1px solid white'}),  # Add a separator line between dropdowns and graphs
    html.Div(id='year-info'),

    dcc.Graph(id='sales-trends-chart'),
    dcc.Graph(id='sales-region-bar-chart'),
    dcc.Graph(id='sales-relationship-chart'),
    dcc.Graph(id='sales-relationship-chart-1'),
    dcc.Graph(id='sales-relationship-chart-2'),
    dcc.Graph(id='sales-pie-chart'),
    html.Hr(style={'border': '1px solid white'}),  # Add a separator line between dropdowns and graphs
    html.Div(id='country-info'),
    html.H1(children='Sales For Selected Country'),
    # Fix the dropdowns syntax
    html.Div([
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
            value=df['year'].max(),
            clearable=False,
            style={'width': '48%', 'color': 'black'}
        ),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': str(geo), 'value': geo} for geo in df['geography'].unique()],
            value=df['geography'].iloc[0],
            clearable=False,
            style={'width': '48%', 'color': 'black'}
        )
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),  # <== Add the comma here!

    html.Hr(style={'border': '1px solid white'}),  # Add a separator line between dropdowns and graphs
    dcc.Graph(id='sales-relationship-chart-3'),
    dcc.Graph(id='sales-relationship-chart-4'),
    dcc.Graph(id='sales-relationship-chart-5'),
    dcc.Graph(id='sales-relationship-chart-6'),
])


# Callbacks to update the dashboard based on the selected year
@app.callback(
    Output('sales-pie-chart', 'figure'),
    Output('sales-trends-chart', 'figure'),
    Output('sales-region-bar-chart', 'figure'),
    Output('sales-relationship-chart', 'figure'),
    Output('sales-relationship-chart-1', 'figure'),
    Output('sales-relationship-chart-2', 'figure'),
    Output('sales-relationship-chart-3', 'figure'),
    Output('sales-relationship-chart-4', 'figure'),
    Output('sales-relationship-chart-5', 'figure'),
    Output('sales-relationship-chart-6', 'figure'),
    Input('year-dropdown', 'value')
    ,Input('country-dropdown', 'value')
)
def update_dashboard(selected_year,selected_country):
    sales_by_region = df.groupby('geography')['total_volume'].sum().reset_index()
    pie_fig = px.pie(sales_by_region, values='total_volume', names='geography',
                 title='Avocado Sales by Region',
                )

    line_monthly_fig = go.Figure()

    line_monthly_fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales.values,
                            mode='lines', name='Monthly Sales'))

    line_monthly_fig.update_layout(title='Monthly Avocado Sales',
                      xaxis_title='Date',
                      yaxis_title='Total Volume')
    # Create subplots for decomposition components
    line_fig = make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residuals'))

    # Add traces for observed data
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.observed, mode='lines', name='Observed'), row=1, col=1)
    # Add traces for trend
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.trend, mode='lines', name='Trend'), row=2, col=1)
    # Add traces for seasonal component
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
    # Add traces for residuals
    line_fig.add_trace(go.Scatter(x=monthly_sales.index, y=decomposition.resid, mode='lines', name='Residuals'), row=4, col=1)

    line_fig.update_layout(height=800, title_text="Time Series Decomposition")

    #Average Price by Season
    season_price = df.groupby('season')['average_price'].mean().reset_index()
    bar_fig = px.bar(season_price, x='season', y='average_price', title='Average Price by Season',
                  labels={'season': 'Season', 'average_price': 'Average Price'})
    #Total Sales by Season
    season_sales = df.groupby('season')['total_volume'].sum().reset_index()
    fig4 = make_subplots(rows=1, cols=2, subplot_titles=('Total Sales by Season', 'Average Price by Season'))
    fig4.add_trace(go.Bar(x=season_sales['season'], y=season_sales['total_volume'], name='Total Sales'), row=1, col=1)
    fig4.add_trace(go.Bar(x=season_price['season'], y=season_price['average_price'], name='Average Price'), row=1, col=2)
    fig4.update_layout(title='Total Sales and Average Price by Season')
    #Volume Price Interaction
    fig5 = make_subplots(rows=1, cols=2, subplot_titles=('Average Volume Price by Promotion','Sales with and without Promotions'))
    volume_price_interaction = df.groupby('promotion')['volume_price_interaction'].mean().reset_index()
    fig5.add_trace(go.Bar(x=volume_price_interaction['promotion'], y=volume_price_interaction['volume_price_interaction'], name='Average Volume Price Interaction'), row=1, col=1)

    promotion_interaction = df.groupby('promotion')['volume_price_interaction'].mean().reset_index()
    fig5.add_trace(go.Bar(x=promotion_interaction['promotion'], y=promotion_interaction['volume_price_interaction'], name='Sales with and without Promotions'), row=1, col=2)
    fig5.update_layout(title='Average Volume Price Interaction by Promotion')

    fig6 = go.Figure()
    fig6.add_trace(go.Line(
        x=df.index,
        y=df['total_volume'],
        name="Sales"
    ))
    fig6.add_trace(go.Line(
        x=df.index,
        y=df['promotion'] * df['total_volume'].max(),
        name="promotion"
    ))
    fig6.update_layout(
        title="Sales and Promotions over Time",title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Sales Volume"
    )
    #
    dff_Ctry=df[df["geography"]==selected_country]      # Filter Countries
    dff_temp_year=dff_Ctry[dff_Ctry["year"]==selected_year]        # filter Year Also selected by user
    dff_temp_year=dff_temp_year.sort_values(by='month')
    fig7 = px.bar(dff_temp_year,x='month', y="monthly_avg_volume")

    fig8 = px.line(dff_temp_year,x='month', y="month_change")
    fig9 = px.bar(dff_Ctry,x='year', y="total_volume")
    fig10 =px.pie(dff_Ctry,values='total_volume',names='weekday_name')
    return pie_fig, line_fig ,line_monthly_fig, fig4, fig5, fig6, fig7, fig8, fig9, fig10

# Run app
if __name__ == '__main__':
    app.run(debug=True)

<ipython-input-4-294d7ca97a32>:21: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



<IPython.core.display.Javascript object>